# Naive Bayes Classifier

In [1]:
import pandas as pd
import numpy as np

# load the dataset
df = pd.read_csv('cancer.csv')

# display the first few rows of the dataset
df.head(2)

,"diagnosis(1=m, 0=b)",radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [2]:
df = df.rename(columns = {'diagnosis(1=m, 0=b)': 'condition'})

In [3]:
import time

start = time.time()


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(['condition'], axis=1), 
                                                    df['condition'], 
                                                    test_size=0.2, 
                                                    random_state=42)

class NaiveBayesClassifier:
    def __init__(self, alpha=1):
        self.alpha = alpha  # Laplace smoothing parameter
        self.classes = None  # list of class labels
        self.class_priors = None  # dictionary of class priors
        self.class_likelihoods = None  # dictionary of class conditional likelihoods
    
    def fit(self, X, y):
        self.classes = list(set(y))
        self.class_priors = {}
        self.class_likelihoods = {}
        
        # calculate class priors
        for c in self.classes:
            self.class_priors[c] = (y == c).sum() / len(y)
        
        # calculate class conditional likelihoods
        for c in self.classes:
            # get the subset of training data belonging to class c
            X_c = X[y == c]
            # calculate the mean and variance of each feature in X_c
            means = X_c.mean(axis=0)
            variances = X_c.var(axis=0) + self.alpha
            # store the mean and variance for class c
            self.class_likelihoods[c] = (means, variances)
    
    def predict(self, X):
        # calculate the log posterior probability of each class for each instance in X
        log_posteriors = []
        for x in X:
            log_p_c = {}
            for c in self.classes:
                # calculate the log prior probability of class c
                log_p_c[c] = np.log(self.class_priors[c])
                # calculate the log likelihood of x given class c
                means, variances = self.class_likelihoods[c]
                log_likelihood = np.sum(-0.5 * np.log(2 * np.pi * variances) - 0.5 * ((x - means) ** 2 / variances), axis=0)
                log_p_c[c] += log_likelihood.sum()
            log_posteriors.append(log_p_c)
        
        # predict the class with the highest log posterior probability for each instance in X
        y_pred = []
        for log_p_c in log_posteriors:
            max_log_p = -np.inf
            max_c = None
            for c, log_p in log_p_c.items():
                if log_p > max_log_p:
                    max_log_p = log_p
                    max_c = c
            y_pred.append(max_c)
        return y_pred
    
#create an instance of the NaiveBayesClassifier class
nb = NaiveBayesClassifier()

#fit the training data
nb.fit(X_train.values, y_train.values)

#make predictions on the testing data
y_pred = nb.predict(X_test.values)

#calculate accuracy
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.2f}')


end = time.time()

print('\n')
print("Time taken to run code in Build code in seconds -- ",end - start)

Accuracy: 0.96


Time taken to run code in Build code in seconds --  0.7892704010009766
